# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

#### Azure Machine Learning and Pipeline SDK-specific imports

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

### Compute cluster

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpucluster4#"#auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True) #min_node_count = 1, , timeout_in_minutes = 10
# For a more detailed view of current AmlCompute status, use get_status().

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

............

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'palsy'
project_folder = './palsy-project'


experiment=Experiment(ws, experiment_name)
print(experiment)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bellpalsy"
description_text = "Datapoints of face"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
#         datastore = Datastore.get(ws, datastore_name='palda')
#         example_data=datastore
        path="https://raw.githubusercontent.com/nsourlos/palda_azure/main/palda.csv"
#         example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(path=path)#path = [(datastore, 'palda.csv')])#example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = df.iloc[: , :-1]

df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
# split df!!!!

x_train,x_test,y_train,y_test=train_test_split(df,shuffle=True)#x,y,test_size=0.2,random_state=42)


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

You can define outputs for the AutoMLStep using TrainingOutput

In [ ]:
# from azureml.pipeline.core import PipelineData, TrainingOutput

# ds = ws.get_default_datastore()
# metrics_output_name = 'metrics_output'
# best_model_output_name = 'best_model_output'

# metrics_data = PipelineData(name='metrics_data',
#                            datastore=ds,
#                            pipeline_output_name=metrics_output_name,
#                            training_output=TrainingOutput(type='Metrics'))
# model_data = PipelineData(name='model_data',
#                            datastore=ds,
#                            pipeline_output_name=best_model_output_name,
#                            training_output=TrainingOutput(type='Model'))

In [ ]:
# automl_step = AutoMLStep(
#     name='automl_module',
#     automl_config=automl_config,
#     outputs=[metrics_data, model_data],
#     allow_reuse=True)

In [ ]:
# from azureml.pipeline.core import Pipeline
# pipeline = Pipeline(
#     description="pipeline_with_automlstep",
#     workspace=ws,    
#     steps=[automl_step])

In [ ]:
# TODO: Put your automl settings here
#taken from https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted',
    "enable_onnx_compatible_models"=True
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=df,
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
remote_run.get_metrics()

import onnxmltools

best_run, fitted_model = remote_run.get_output(return_onnx_model=True)
# Taken from https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

onnxmltools.utils.save_model(fitted_model, './outputs/best_automl.onnx')

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_name = "palda_dataset",
                        model_path = "./outputs/best_automl.onnx",
                        description = "Best AutoML Model",)

In [ ]:
#taken from https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py
#and from https://stackoverflow.com/questions/66437607/how-to-create-azure-ml-inference-config-and-deployment-config-class-objects-from

from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')




from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               description = 'palsy classification service')

aci_service_name = 'palsy classification'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)



# InferenceConfig(entry_script=)


In [ ]:
# published_pipeline = remote_run.publish_pipeline(
#     name="palda", description="palsy dataset", version="1.0")

# published_pipeline

In [ ]:
# from azureml.core.authentication import InteractiveLoginAuthentication

# interactive_auth = InteractiveLoginAuthentication()
# auth_header = interactive_auth.get_authentication_header()



In [ ]:
# import requests

# rest_endpoint = published_pipeline.endpoint
# response = requests.post(rest_endpoint, 
#                          headers=auth_header, 
#                          json={"ExperimentName": "pipeline-rest-endpoint"}
#                         )

In [ ]:
# try:
#     response.raise_for_status()
# except Exception:    
#     raise Exception("Received bad response from the endpoint: {}\n"
#                     "Response Code: {}\n"
#                     "Headers: {}\n"
#                     "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

# run_id = response.json().get('Id')
# print('Submitted pipeline run: ', run_id)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
print("Key: " + aci_service.get_keys()) #[0]


In [ ]:
change data from test file!!!!

In [ ]:
#endpoint.py file
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
# scoring_uri = 'http://7bbc8877-700c-4035-8097-c35c896039a2.southcentralus.azurecontainer.io/score'
scoring_uri=aci_service.scoring_uri
# If the service is authenticated, set the key or token
# key = 'Y9bgMCp5O0geM74pEThdQVLiaRMHBjrB'
key=aci_service.get_keys()[0]

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "age": 17,
            "job": "blue-collar",
            "marital": "married",
            "education": "university.degree",
            "default": "no",
            "housing": "yes",
            "loan": "yes",
            "contact": "cellular",
            "month": "may",
            "day_of_week": "mon",
            "duration": 971,
            "campaign": 1,
            "pdays": 999,
            "previous": 1,
            "poutcome": "failure",
            "emp.var.rate": -1.8,
            "cons.price.idx": 92.893,
            "cons.conf.idx": -46.2,
            "euribor3m": 1.299,     
            "nr.employed": 5099.1,      
          },
          {
            "age": 87,
            "job": "blue-collar",
            "marital": "married",
            "education": "university.degree",
            "default": "no",
            "housing": "yes",
            "loan": "yes",
            "contact": "cellular",
            "month": "may",
            "day_of_week": "mon",
            "duration": 471,
            "campaign": 1,
            "pdays": 999,
            "previous": 1,
            "poutcome": "failure",
            "emp.var.rate": -1.8,
            "cons.price.idx": 92.893,
            "cons.conf.idx": -46.2,
            "euribor3m": 1.299, 
            "nr.employed": 5099.1,   
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
#print(resp)
print(resp.json())


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
aci_service.get_logs()

In [ ]:
#logs.py file
from azureml.core.webservice import Webservice

# Set with the deployment name
name = "palsy"

# load existing web service
service = Webservice(name=name, workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

service.update(enable_app_insights=True)

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.


In [ ]:
#aci_service.delete()

# service.delete()

# compute_cluster.delete()
